In [ ]:
from json.decoder import JSONDecodeError
import subprocess
import json
import base64
import socket
import os
 
#--------------Connect---------------
HOST = ""  
PORT = 5555

print("Waiting for connection...")
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((HOST, PORT))
s.listen()
conn, addr = s.accept()
conn.setblocking(0)

print('Connected by', addr)
connected = True
#-------------------------------------

#-------------Run Base Function---------------
ColorTracker = subprocess.Popen(['/home/m5stack/payload/bin/color_tracker'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
ColorTracker.stdin.write("_{\"stream\":1}\r\n".encode('utf-8'))
ColorTracker.stdin.flush()
#---------------------------------------------

img = ""
cx = "0"
cy = "0"
r = "0"
mx = "0"
my = "0"


def ColorTracker_getROI(dataIn):
    ROI = dataIn.split(",")
    try:
        if ROI[0] == 'ROI':
            roiX = ROI[1]
            roiY = ROI[2]
            roiW = ROI[3]
            roiH = ROI[4]
            print("Received ROI: ", roiX, " ", roiY, " ", roiW, " ", roiH)
            ColorTracker_SetROI(int(roiX), int(roiY), int(roiW), int(roiH)) 
    except Exception as e: 
        print(e)
        pass
    
def ColorTracker_getLAB(dataIn):
    LAB = dataIn.split(",")
    try:
        if LAB[0] == 'LAB':
            L_min = LAB[1]
            L_max = LAB[2]
            A_min = LAB[3]
            A_max = LAB[4]
            B_min = LAB[5]
            B_max = LAB[6]
            print("Received LAB: ", L_min, " ", L_max, " ", A_min, " ", A_max, " ", B_min, " ", B_max)
            ColorTracker_SetLAB(int(L_min), int(L_max), int(A_min), int(A_max), int(B_min), int(B_max))
    except Exception as e: 
        print(e)
        pass
    
def ColorTracker_SetROI(x, y, w, h):
    roi = { 
         'config':"Color Tracker",
              'x': x,
              'y': y,
              'w': w,
              'h': h
        }
    roi = json.dumps(roi)
    roi = roi.encode('utf-8')
    
    ColorTracker.stdin.write(roi)
    ColorTracker.stdin.flush()
    ColorTracker.stdin.write("_{\"stream\":1}\r\n".encode('utf-8'))
    ColorTracker.stdin.flush()

def ColorTracker_SetLAB(l_min, l_max, a_min, a_max, b_min, b_max):
    lab = {
        "config":"Color Tracker",
        "l_min": l_min, # 0 ~ 255
        "l_max": l_max, # 0 ~ 255
        "a_min": a_min, # 0 ~ 255
        "a_max": a_max, # 0 ~ 255
        "b_min": b_min, # 0 ~ 255
        "b_max": b_max  # 0 ~ 255
    } 
    lab = json.dumps(lab)
    lab = lab.encode('utf-8')
    
    ColorTracker.stdin.write(lab)
    ColorTracker.stdin.flush()
    ColorTracker.stdin.write("_{\"stream\":1}\r\n".encode('utf-8'))
    ColorTracker.stdin.flush()
    
def ColorTracker_CheckCoordinates(doc):
    global cx, cy, r, mx, my
    if ('cx' in doc) and ('cy' in doc) and ('r' in doc) and ('mx' in doc)  and ('my' in doc):
        cx = str(doc['cx'])
        cy = str(doc['cy'])
        r = str(doc['r'])
        mx = str(doc['mx'])
        my = str(doc['mx'])
    else:
        cx = "0"
        cy = "0"
        r = "0"
        mx = "0"
        my = "0"
#     print("Target: ", cx, " ", cy, " ", r, " ", mx,  " ", my) 

def ColorTracker_CheckImg(doc):
    global img
    if ('img' in doc):
        img = doc['img']
        
def ColorTracker_SendUpdate():
    global conn
    conn.sendall(cx.encode())
    conn.sendall(','.encode())
    conn.sendall(cy.encode())
    conn.sendall(','.encode())
    conn.sendall(r.encode())
    conn.sendall(','.encode())
    conn.sendall(mx.encode())
    conn.sendall(','.encode())
    conn.sendall(my.encode())
    conn.sendall(','.encode())
    conn.sendall(img.encode())
    conn.sendall('\n'.encode())

while True:
    try:
        doc = json.loads(ColorTracker.stdout.readline().decode('utf-8')) 

        ColorTracker_CheckCoordinates(doc)
        ColorTracker_CheckImg(doc)   

        if connected:
            try:
                ColorTracker_SendUpdate()
            except Exception as e: 
                if e.errno == 32 or e.errno == 104:
                    print("Connection lost")
                    conn.detach();
                    connected = False
        else:
            print("Waiting for connection...")
            conn, addr = s.accept()
            conn.setblocking(0)
            print('Connected by', addr) 
            connected = True
            
    except Exception as e: 
        print(e)
        pass
    
    try:
        dataIn = conn.recv(1024)
        dataIn = dataIn.decode('utf-8')
        ColorTracker_getROI(dataIn)
        ColorTracker_getLAB(dataIn)
    except Exception as e: 
        pass